In [4]:
import pymysql
from sshtunnel import SSHTunnelForwarder

# Define your SSH and MySQL credentials
ssh_host = '10.120.21.103'
ssh_port = 22
ssh_user = 'setrag'
ssh_password = 'Wabjtam,123'

mysql_host = '192.168.21.69'
mysql_port = 3306
mysql_user = 'galera'
mysql_password = 'galera'
mysql_db = 'setrag_beta1'

# Establish the SSH tunnel
with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_user,
    ssh_password=ssh_password,
    remote_bind_address=(mysql_host, mysql_port)
) as tunnel:
    # Connect to the MySQL database
    connection = pymysql.connect(
        host='127.0.0.1',
        user=mysql_user,
        password=mysql_password,
        db=mysql_db,
        port=tunnel.local_bind_port
    )

    # Execute a query
    with connection.cursor() as cursor:
        cursor.execute("""
                        SELECT t.number AS 'N° de train',
                            tp.label AS 'Type de train',
                            count(sbr.seat_occupation_flag) AS 'Places disponibles'
                        FROM train AS t
                        INNER JOIN seat_booking_record AS sbr
                        ON sbr.train_id = t.id
                        INNER JOIN seat AS s
                        ON s.id = sbr.seat_id
                        INNER JOIN car AS c
                        ON c.id = s.car_id
                        INNER JOIN train_type tp
                        ON t.train_type_id = tp.id
                        WHERE (t.planned_date/1000) between UNIX_TIMESTAMP('2024-12-01 00:00:00') AND UNIX_TIMESTAMP('2024-12-31 23:59:59') 
                        AND sbr.seat_occupation_flag = 0 
                        AND tp.label IN ('EXPRESS', 'OMNIBUS') 
                        GROUP BY DATE_FORMAT(FROM_UNIXTIME(t.planned_date/1000), '%d/%m/%Y'), t.number
        """)
        result = cursor.fetchall()
        field_names = [i[0] for i in cursor.description]
        print(field_names)
        for row in result:
            print(row)

    # Close the connection
    connection.close()

['N° de train', 'Type de train', 'Places disponibles']
('712', 'EXPRESS', 50)
('731', 'OMNIBUS', 37)
('211', 'EXPRESS', 26)
('412', 'EXPRESS', 7)
('431', 'OMNIBUS', 2)
('611', 'EXPRESS', 77)
('632', 'OMNIBUS', 77)
('712', 'EXPRESS', 58)
('731', 'OMNIBUS', 102)
('211', 'EXPRESS', 133)
('232', 'OMNIBUS', 162)
('412', 'EXPRESS', 187)
('431', 'OMNIBUS', 149)
('611', 'EXPRESS', 382)
('632', 'OMNIBUS', 186)
('712', 'EXPRESS', 90)
('731', 'OMNIBUS', 238)
('211', 'EXPRESS', 298)
('232', 'OMNIBUS', 67)
('412', 'EXPRESS', 145)
('431', 'OMNIBUS', 78)
('611', 'EXPRESS', 197)
('632', 'OMNIBUS', 84)
('712', 'EXPRESS', 160)
('731', 'OMNIBUS', 43)
('211', 'EXPRESS', 242)
('232', 'OMNIBUS', 108)
('412', 'EXPRESS', 233)
('431', 'OMNIBUS', 168)
('611', 'EXPRESS', 299)
('632', 'OMNIBUS', 105)
('712', 'EXPRESS', 274)
('731', 'OMNIBUS', 105)
